In [5]:
from src.lexer import Lexer, TokenType

with open("tests/lexer.txt", "r") as f:
    code = f.read()
    
print(code)

// Commentaire
/*
    Multi-line
    comment
*/
123 4.7
"hello world" "hello \nworld"
x some_variable
none not and or true false function return if else for while
== != <= >= **
+ - * / % = < > ( ) { } [ ] , ; : .
+= -= *= /= %= **=


In [6]:
code = "4. 1.2"

In [7]:
lexer = Lexer(code)
tokens = lexer.tokenize()
for token in tokens:
    if token.type == TokenType.NEWLINE:
        print("\n")
        continue
    print(token)

Token(type=<TokenType.FLOAT>, value='4.', line=1, column=1)
Token(type=<TokenType.FLOAT>, value='1.2', line=1, column=4)
Token(type=<TokenType.EOF>, value='', line=1, column=7)
